# Gridworld

In [1]:
# Find the value function of policy
import numpy as np

# display output
from random import uniform
import time
from IPython.display import display, clear_output

In [2]:
actions = [[-1, 0], [0, 1], [1, 0], [0, -1]] #up, right, down, left = (clockwise from up) 
action_count = len(actions) # total number of actions
gridSize = 5 # create a square grid of gridSize by gridSize
state_count = gridSize*gridSize # total number of states

In [3]:
class Gridworld():
    def __init__(self, gridSize):
        self.valueMap = np.zeros((gridSize, gridSize))
        self.states = [[i, j] for i in range(gridSize) for j in range(gridSize)]
        self.size = gridSize
        self.new_pos = [0, 0] # initialize new position for p_transition
        self.transition_prob = 1 # deterministic
    
    def initial_state(self):        # return initial state
        return grid.states[gridSize*gridSize-1] # ? self.states
   
    def transition_reward(self, current_pos, action): # return the transition probability

        # get next position: state: [0, 0], action: [0, 1], new_state = [0, 1]
        self.new_pos = np.array(current_pos) + np.array(action)

        # normally, reward = 0
        reward = 0

        # if new pos results in off the grid, return reward -1
        if -1 in self.new_pos or self.size in self.new_pos:
            reward = -1
        # if in state A, receive + 10
        if current_pos == [0, 1]:
            reward = 10
        # if in state B, receive + 5
        if current_pos == [0, 3]:
            reward = 5

        # if taking an action crosses the border; agent's new_pos is the same as the current pos
        if -1 in self.new_pos or self.size in self.new_pos: 
            self.new_pos = current_pos
            
        # if in state A, transition to state A'
        if current_pos == [0, 1]:
            self.new_pos = [4, 1]
            
        # if in state B, transition to state B'
        if current_pos == [0, 3]:
            self.new_pos = [2, 3]

        return self.new_pos, reward

In [4]:
# create a grid object
grid = Gridworld(5)

In [5]:
# get initial state (bottom right)
grid.initial_state()

[4, 4]

Initialize a random policy

In [6]:
# Initiate a random policy
random_policy = np.random.randint(1000, size=(state_count, action_count))
random_policy = random_policy/random_policy.sum(axis=1)[:,None]
policy = random_policy

In [7]:
# random policy
policy

array([[0.38041667, 0.39083333, 0.03916667, 0.18958333],
       [0.05014191, 0.46736045, 0.09176916, 0.39072848],
       [0.26320611, 0.26748092, 0.25648855, 0.21282443],
       [0.34566929, 0.29448819, 0.12637795, 0.23346457],
       [0.15637394, 0.39433428, 0.07422096, 0.37507082],
       [0.18901454, 0.18309101, 0.34841142, 0.27948304],
       [0.37226579, 0.29137433, 0.14156005, 0.19479983],
       [0.39586028, 0.20137991, 0.34928849, 0.05347132],
       [0.36378335, 0.18108327, 0.19401778, 0.2611156 ],
       [0.01147626, 0.24308816, 0.35367762, 0.39175796],
       [0.26828087, 0.12445521, 0.48184019, 0.12542373],
       [0.        , 0.07354346, 0.25023878, 0.67621777],
       [0.2837345 , 0.35010941, 0.26477024, 0.10138585],
       [0.50028265, 0.10457886, 0.30525721, 0.08988129],
       [0.33645346, 0.2324493 , 0.0400416 , 0.39105564],
       [0.18852459, 0.10403531, 0.44199243, 0.26544767],
       [0.11250983, 0.27458694, 0.3859166 , 0.22698662],
       [0.37138085, 0.09242762,

### Create an Episode following policy

In [8]:
def generate_episode(steps):

    # set initial state
    state = grid.initial_state()

    # initialize state (with iniitial state), action list and reward list
    state_list = [state]
    action_list = []
    reward_list = []

    # generate an episode
    for i in range(steps):

        # pick an action based on categorical distribution in policy
        action = int(np.random.choice(action_count, 1, p=policy[grid.states.index(state)])) # get index in int
        action = actions[action] # convert the integer index (ie. 0) to action (ie. [-1, 0])

        # get reward (integer value)
        reward = grid.reward(state, action)

        # get the new state with the chosen action
        new_state = list(grid.p_transition(state, action)) # (ie. [1,0])
        state = new_state # set the next state as the current state

        # save state, action chosen and reward to list
        state_list.append(state)
        action_list.append(action)
        reward_list.append(reward)
        
    return state_list, action_list, reward_list

### SARSA(Lamda)

In [15]:
# initialize q values for all state action pairs
Q_values = np.random.randint(0,1000,size = (state_count, action_count))
Q_values

array([[496, 639, 662, 911],
       [832, 815,  31, 849],
       [831, 851, 231, 657],
       [701, 334,   1, 774],
       [190, 555, 741, 142],
       [603, 433, 207,  23],
       [945,  74, 536, 121],
       [643, 878, 840, 521],
       [912, 163, 805, 413],
       [562,  50, 226, 912],
       [528, 711, 392, 406],
       [112, 130, 818, 689],
       [818, 113,  80, 425],
       [513, 735, 885, 385],
       [998, 984, 749, 695],
       [628, 802, 891,  94],
       [840, 846,  26, 183],
       [917,  83, 653,  43],
       [568, 837, 744, 997],
       [ 26,  15,  27,  77],
       [ 53, 460, 740, 778],
       [824,   3, 406, 913],
       [943,  72, 245, 666],
       [863, 794, 995, 153],
       [110, 198, 300, 123]])

In [10]:
# intialize parameters
gamma = 0.99
epsilon = 0.2
lamda = 0.9
alpha = 0.1

In [11]:
# iterate 500 times: each time, generating an episode of 200 steps
max_steps = 200

# define variables for keeping track of time steps
Terminal = max_steps
t_list=[]
for i in range(1,max_steps+1):
    t = Terminal - i
    t_list.append(t)

In [12]:
def choose_action(state, epsilon):
    
    # choose an action type: explore or exploit
    action_type = int(np.random.choice(2, 1, p=[epsilon,1-epsilon]))

    # find best action based on Q values
    best_action_index = np.argmax(Q_values[state])

    # choose an action based on exploit or explore
    if action_type == 0:
        # explore
        # print("explore")
        
        # pick a random action
        random_action_index = np.random.choice(range(4))
        
        # while random action is the same as the best action, pick a new action
        while random_action_index == best_action_index:
            random_action_index = np.random.choice(range(4))
        
        action_index = random_action_index
    else:
        # exploit
        # print("exploit")
        action_index = best_action_index
        
    return action_index

In [25]:
action_type = int(np.random.choice(2, 1, p=[epsilon,1-epsilon]))
action_type

0

In [21]:
# iteration 500 times
for iteration in range(500):
    
    # initialize delta
    delta = 0
    
    # initialize S,A (? should i choose an Action using epsilon-greedy here or just select an Action?)
    state_vector = grid.initial_state()
    state_index = grid.states.index(state_vector)
    
    # initialize  eligibility traces for all state action pairs of all states to 0
    z_values = np.zeros((state_count, action_count))
    
    action_index = choose_action(state_index, epsilon)
    action_vector = actions[action_index]
    
    # iteration 200 steps of the episode
    for i in range(max_steps):
        
        # Take action A, oberserve R, S'
        next_state_vector, reward = grid.transition_reward(state_vector, action_vector)
        next_state_index = grid.states.index(list(next_state_vector))
        
        # Choose A' from S' using policy derived from Q (eg. epsilon-greedy)
        next_action_index = choose_action(next_state_index, epsilon)
        next_action_vector = actions[next_action_index]
        
        # update the action-value form of the TD error
        delta = reward + gamma*Q_values[next_state_index][next_action_index] - Q_values[state_index][action_index]
        
        # accumulate traces (? big S and big A?)
        z_values[state_index][action_index] +=1
        
        # update Q value
        Q_values[state_index][action_index] = Q_values[state_index][action_index] + alpha*delta*z_values[state_index][action_index]
        
        # update z value
        z_values[state_index][action_index] = gamma*lamda*z_values[state_index][action_index]
        
        # update state and action vector
        state_vector = list(next_state_vector)
        state_index = grid.states.index(state_vector)
        action_vector = list(next_action_vector)
        action_index = next_action_index

In [14]:
np.set_printoptions(precision=2)
Q_values

array([[ 3,  9,  1,  0],
       [12, 10, 10, 10],
       [ 3,  4,  0,  9],
       [12,  5,  6,  5],
       [ 2,  1,  0,  4],
       [ 6,  4,  0,  0],
       [ 9,  2,  0,  1],
       [ 5,  0,  0,  2],
       [ 6,  0,  0,  1],
       [ 1,  0,  0,  0],
       [ 4,  0,  0,  0],
       [ 1,  0,  0,  1],
       [ 2,  0,  0,  1],
       [ 6,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 1,  0,  0,  0],
       [ 7,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 1,  0,  1,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0]])

In [16]:
# PRINT POLICY TABLE ################################################################################
# import pandas library
import pandas as pd
# define column and index
columns=range(grid.size)
index = range(grid.size)
# define dataframe to represent policy table
policy_table = pd.DataFrame(index = index, columns=columns)

# iterate through policy to make a table that represents action number
# as action name (eg. left, right, up, down)
for state in range(len(Q_values)):
    
    # find the best action at each state
    best_action = np.argmax(Q_values[state])

    # get action name
    if best_action == 0:
        action_name = 'up'
    elif best_action == 1:
        action_name = 'right'
    elif best_action == 2:
        action_name = 'down'
    else:
        action_name = 'left'

    # calculate the row and column coordinate of the current state number
    row = int(state/grid.size)
    column = round((state/grid.size - int(state/grid.size))*grid.size)
            
    # assign action name
    policy_table.loc[row][column] = action_name

print("Policy Table: ")
print(policy_table)
print()

Policy Table: 
       0      1      2     3     4
0   left   left  right  left  down
1     up     up  right    up  left
2  right   down     up  down    up
3   down  right     up  left  left
4   left   left     up  down  down

